# 微博情感分析

In [89]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
import torch.nn.functional as F 
import time
from tqdm import tqdm
import torchtext.vocab as Vocab
import argparse
import os
import pandas as pd
import numpy as np
import collections
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import datetime

## 数据介绍

0喜悦
1愤怒
2厌恶
3低落

In [2]:
dictionary=np.load("dictionary.npy",allow_pickle=True)
#torch.set_default_tensor_type(torch.FloatTensor)

In [3]:
def labels_encode():
    y=[int(line) for line in open("labels.txt", encoding="utf-8").readlines()]
    y=np.expand_dims(np.array(y),1)
    encoder=OneHotEncoder()
    return encoder.fit_transform(y).toarray()

In [4]:
y=labels_encode()

In [5]:
def texts_encode(X):
    for i in range(len(X)):
        X[i]=[np.int64(dictionary.item()[j]) for j in X[i]]
    return X


In [39]:
X=[line.split() for line in open("texts.txt",encoding="utf-8").readlines()]

In [40]:
X=texts_encode(X)

In [41]:
def cut(X,length):
    for key,value in enumerate(X):
        if len(value)<=length:
            X[key]=value+[0 for i in range (length-len(value))]
        else:
            X[key]=X[key][:length]
    return X

In [42]:
X=np.array(cut(X,20))

In [67]:
class Rnn_simple(nn.Module):
    def __init__(self,vocab,embed_size,num_hiddens,num_layers):
        super(Rnn_simple,self).__init__()
        self.embedding=nn.Embedding((vocab),embed_size)
        self.encoder=nn.LSTM(input_size=embed_size,hidden_size=num_hiddens,num_layers=num_layers,bidirectional=True)
        self.decoder=nn.Linear(4*num_hiddens,4)
    def forward(self,X):
        X=self.embedding(X.permute(1,0))
        X,_=self.encoder(X)
        X=torch.cat((X[0],X[-1]),-1)
        X=self.decoder(X)
        return X

In [76]:
vocab,embed_size,num_hiddens,num_layers=len(dictionary.item())+1,100,100,2

In [77]:
net=Rnn_simple(vocab,embed_size,num_hiddens,num_layers)

In [78]:
net

Rnn_simple(
  (embedding): Embedding(24759, 100)
  (encoder): LSTM(100, 100, num_layers=2, bidirectional=True)
  (decoder): Linear(in_features=400, out_features=4, bias=True)
)

In [79]:
X=torch.LongTensor(X)

tensor([[-0.0009, -0.0294,  0.0336, -0.0037],
        [-0.0116,  0.0006,  0.0382, -0.0136],
        [-0.0135,  0.0070,  0.0373, -0.0358],
        ...,
        [-0.0202, -0.0179,  0.0555, -0.0178],
        [-0.0383, -0.0116,  0.0644, -0.0483],
        [ 0.0044, -0.0294,  0.0552, -0.0102]], grad_fn=<AddmmBackward>)

In [ ]:
    def evaluate(valdata,vallabel):
        ans=0.;
        with torch.no_grad():
            prelabel=network(valdata)
            for i in range(len(prelabel)):
                if prelabel[i][0]>=0.5 and int(vallabel[i][0])==1:
                    ans+=1
                if prelabel[i][0]<0.5 and int(vallabel[i][0])==0:
                    ans+=1
        print("the accuracy is",ans/float(len(prelabel)))  

In [84]:

def trainloop( n_epochs,dataloader,network,optim,loss_fn):
    for epoch in range(1, n_epochs+1):
        #evaluate(mask_data.data,mask_data.label)
        loss_train = 0.0
        for input, realout in dataloader:
            predictout = network(input)

            loss = loss_fn(predictout, realout)

            optim.zero_grad()

            loss.backward()
            optim.step()
            loss_train += loss.item()
            #if epoch == 1 or epoch % 100 == 0:
        print(
                f'{datetime.datetime.now()} epoch {epoch} training loss {loss_train/len(dataloader)}')

In [87]:
n_epochs=50
network=net
optim=torch.optim.Adam(network.parameters(),lr=0.001)
dataset=TensorDataset(X,torch.Tensor(y))
dataloader=DataLoader(dataset,batch_size=300,shuffle=True)
loss_fn=nn.BCEWithLogitsLoss()

In [88]:
trainloop(n_epochs,dataloader,network,optim,loss_fn)

NameError: name 'datetime' is not defined